In [ ]:
!pip install avalanche-lib medmnist

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.2/134.2 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.4/993.4 kB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.2/585.2 kB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 65.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 630.5/630.5 kB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.1/222.1 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.4/172.4 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 90.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 74.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 85.2 MB/s eta 0:00:00
  

In [ ]:
#Imports
import torch
from torch import nn, optim
from torch.utils.data import Dataset

from torchvision import transforms
from medmnist import PathMNIST

from avalanche.benchmarks import nc_benchmark
from avalanche.training import Replay
from avalanche.evaluation.metrics import (
    accuracy_metrics,
    forgetting_metrics
)
from avalanche.logging import InteractiveLogger
from avalanche.training.plugins import EvaluationPlugin

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

Using device: cuda


In [ ]:
#Dataset wrapper
class MedMNISTWrapper(Dataset):
    def __init__(self, medmnist_dataset):
        self.dataset = medmnist_dataset
        self.targets = [
            int(label) for label in medmnist_dataset.labels.squeeze()
        ]

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        x, y = self.dataset[index]
        return x, int(y)

In [ ]:
#Load PathMNIST
transform = transforms.Compose([
    transforms.ToTensor(),
])

train_raw = PathMNIST(
    split="train",
    transform=transform,
    download=True
)

test_raw = PathMNIST(
    split="test",
    transform=transform,
    download=True
)

train_dataset = MedMNISTWrapper(train_raw)
test_dataset  = MedMNISTWrapper(test_raw)


100%|██████████| 206M/206M [00:36<00:00, 5.61MB/s]


In [ ]:
#Create benchmark
benchmark = nc_benchmark(
    train_dataset,
    test_dataset,
    n_experiences=3,
    task_labels=False,
    seed=1234
)

print("Total classes:", benchmark.n_classes)

Total classes: 9


In [ ]:
#CNN model
class CNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.classifier = nn.Linear(64 * 7 * 7, num_classes)

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        return self.classifier(x)

In [ ]:
eval_plugin = EvaluationPlugin(
    accuracy_metrics(epoch=True, experience=True, stream=True),
    forgetting_metrics(experience=True, stream=True),
    loggers=[InteractiveLogger()]
)

In [ ]:
model = CNN(num_classes=benchmark.n_classes).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

cl_strategy = Replay(
    model,
    optimizer,
    criterion,
    mem_size=2000,        # replay buffer
    train_mb_size=64,
    train_epochs=8,
    eval_mb_size=64,
    evaluator=eval_plugin,
    device=device
)

/usr/local/lib/python3.12/dist-packages/avalanche/training/templates/base.py:468: PositionalArgumentsDeprecatedWarning: Avalanche is transitioning to strategy constructors that accept named (keyword) arguments only. This is done to ensure that there is no confusion regarding the meaning of each argument (strategies can have many arguments). Your are passing 3 positional arguments to the Replay.__init__ method. Consider passing them as names arguments. The ability to pass positional arguments will be removed in the future.
  warnings.warn(error_str, category=PositionalArgumentsDeprecatedWarning)


In [ ]:
for experience in benchmark.train_stream:
    print(f"\nTraining on experience {experience.current_experience}")
    cl_strategy.train(experience)

    print("Evaluating...")
    cl_strategy.eval(benchmark.test_stream)


Training on experience 0
-- >> Start of training phase << --
0it [00:00, ?it/s]

/tmp/ipython-input-1271679127.py:14: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return x, int(y)


100%|██████████| 411/411 [00:07<00:00, 52.19it/s]
Epoch 0 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.8164
100%|██████████| 411/411 [00:06<00:00, 62.79it/s]
Epoch 1 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.8915
100%|██████████| 411/411 [00:06<00:00, 67.89it/s]
Epoch 2 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9179
100%|██████████| 411/411 [00:06<00:00, 59.20it/s]
Epoch 3 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9339
100%|██████████| 411/411 [00:06<00:00, 68.22it/s]
Epoch 4 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9453
100%|██████████| 411/411 [00:06<00:00, 62.03it/s]
Epoch 5 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9546
100%|██████████| 411/411 [00:06<00:00, 63.50it/s]
Epoch 6 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9585
100%|██████████| 411/411 [00:06<00:00, 61.76it/s]
Epoch 7 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9591


/usr/local/lib/python3.12/dist-packages/avalanche/training/plugins/replay.py:123: DeprecationWarning: Call to deprecated function update (removal in version 0.7: switch to pre_adapt and post_adapt)
  self.storage_policy.update(strategy, **kwargs)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


-- >> End of training phase << --
Evaluating...
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
 32%|███▏      | 11/34 [00:00<00:00, 66.72it/s]

...

100%|██████████| 34/34 [00:00<00:00, 78.51it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	ExperienceForgetting/eval_phase/test_stream/Task000/Exp000 = 0.3924
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.4818
-- Starting eval on experience 1 (Task 0) from test stream --
100%|██████████| 42/42 [00:00<00:00, 82.02it/s]
> Eval on experience 1 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp001 = 0.9566
-- Starting eval on experience 2 (Task 0) from test stream --
100%|██████████| 38/38 [00:00<00:00, 77.59it/s]
> Eval on experience 2 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp002 = 0.0000
-- >> End of eval phase << --
	StreamForgetting/eval_phase/test_stream = 0.3924
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.4979

Training on experience 2
-- >> Start of training phase << --
100%|██████████| 456/456 [00:12<00:00, 37.43it/s]
Epoch 0 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000

/usr/local/lib/python3.12/dist-packages/avalanche/training/plugins/replay.py:123: DeprecationWarning: Call to deprecated function update (removal in version 0.7: switch to pre_adapt and post_adapt)
  self.storage_policy.update(strategy, **kwargs)



Epoch 7 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9973
-- >> End of training phase << --
Evaluating...
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
 44%|████▍     | 15/34 [00:00<00:00, 75.70it/s]

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/tmp/ipython-input-1271679127.py:14: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return x, int(y)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/_utils/pin_memory.py:57: DeprecationWarning: The argument 'device' of Tensor.pin_memory() is deprecated. Please do not pass this argument. (Triggered internally at /pytorch/aten/src/ATen/native/Memory.cpp:46.)
  return data.pin_memory(device)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/_utils/pin_memory.py:57: DeprecationWarning: 

100%|██████████| 34/34 [00:00<00:00, 82.73it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	ExperienceForgetting/eval_phase/test_stream/Task000/Exp000 = 0.2459
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.6284
-- Starting eval on experience 1 (Task 0) from test stream --
100%|██████████| 42/42 [00:00<00:00, 74.06it/s]
> Eval on experience 1 (Task 0) from test stream ended.
	ExperienceForgetting/eval_phase/test_stream/Task000/Exp001 = 0.1849
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp001 = 0.7717
-- Starting eval on experience 2 (Task 0) from test stream --
100%|██████████| 38/38 [00:00<00:00, 80.80it/s]
> Eval on experience 2 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp002 = 0.9093
-- >> End of eval phase << --
	StreamForgetting/eval_phase/test_stream = 0.2154
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.7753


In [ ]:
import json

metrics = cl_strategy.evaluator.get_all_metrics()

with open("replay_metrics.json", "w") as f:
    json.dump(metrics, f)